# Adding a new source term: Thermal conduction

In [ ]:
## Import relevant libraries ## 
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
from nm_lib import nm_lib as nm
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# plt.style.use('dark_background')
plt.style.use('default')

Expand the HD equations by adding thermal conduction: 

$ \frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \bf u) = 0 $

$ \frac{\partial \rho {\bf u}}{\partial t} + \nabla \cdot (\rho {\bf u}  \otimes {\bf u}) = - \nabla (P_g)$

$ \frac{\partial e}{\partial t } = -\nabla\cdot e {\bf u} -P_g \nabla \cdot {\bf u} + \nabla \kappa \nabla T_g $

Use the ideal equation of state: 

$P_g = n K_B T_g$

where $n$ is the number of particles and assume everything is hydrogen. $K_B$ is the Boltzman constant. $\kappa$ is the thermal coefficient. 

Note that this term is diffusive. Which numerical scheme would you use to avoid stiffness? 

In [ ]:
# def drho_dt(uu, xx, rho): 
#     ddx_rho = nm.deriv_upw()

def deriv_cent(xx, hh, **kwargs):
    r"""
    returns the centered 2nd derivative of hh respect to xx. 

    Parameters
    ---------- 
    xx : `array`
        Spatial axis. 
    hh : `array`
        Function that depends on xx. 

    Returns
    -------
    `array`
        The centered 2nd order derivative of hh respect to xx. First 
        and last grid points are ill calculated. 
    """
    
    if kwargs['axis']: 
        return (np.roll(hh, -1, axis=kwargs['axis']) - np.roll(hh, 1, axis=kwargs['axis'])) \
            / (np.roll(xx, -1, axis=kwargs['axis']) - np.roll(xx, 1, axis=kwargs['axis']))
    else:
        return (np.roll(hh, -1) - np.roll(hh, 1)) / (np.roll(xx, -1) - np.roll(xx, 1))


def step_density(xx, rho, u, axis=0, cfl_cut=0.98, ddx=lambda x, y: nm.deriv_cent(x, y), bnd_limits=[1,1]): 
    """ 
    Right hand side of the 1D continuity equation, where rho can be a constant or a function of xx. 
    """
    dx = xx[1] - xx[0]
    a = rho*u
    dt = cfl_cut*np.min(dx/np.abs(a))

    return dt, -ddx(xx, a, axis=axis)

def step_momentum(xx, rho, u, Pg, axis=0, cfl_cut=0.98, ddx=lambda x, y: nm.deriv_cent(x, y), bnd_limits=[1,1]): 
    """ 
    Right hand side of the 1D momentm equation, where Pg can be a constant or a function of xx. 
    """
    dx = xx[1] - xx[0]
    a = rho*u**2 + Pg
    # can add term for gravity (- rho*g)
    dt = cfl_cut*np.min(dx/np.abs(a))

    return dt, -ddx(xx, a, axis=axis)

def step_energy(xx, e, u, Pg, axis=0, cfl_cut=0.98, ddx=lambda x, y: nm.deriv_cent(x, y), bnd_limits=[1,1]
                , source_term=None):
    """ 
    Right hand side of the 1D energy equation, where rho can be a constant or a function of xx. 
    """
    dx = xx[1] - xx[0]
    if source_term: 
        a = u*(e + Pg) + source_term
    else: 
        a = u*(e + Pg)
    dt = cfl_cut*np.min(dx/np.abs(a))

    return dt, -ddx(xx, a, axis=axis)

<span style="color:pink">

Here we have a ner part in the energy equation. This would run quite slow using Lax. Therefore we consider running the different parts of the energy equation separatly, and then adding them using operator splitting. 

We begin by writing a code for the hydro-dynamical PDE models. 

The RHS of the continuity equation and for the momentum equation remain the same: 

$$
\frac{\partial\rho}{\partial t} + \frac{\partial\rho u}{\partial x} \rightarrow \frac{\partial\rho}{\partial t} = -\frac{\partial\rho u}{\partial x}
$$

$$
\frac{\partial\rho u}{\partial t} = -\frac{\partial\rho u^2}{\partial x} - \frac{\partial P g}{\partial x}. 
$$

While the energy equation should be rewritten to include the source term: 

$$
\frac{\partial e}{\partial t} = -\frac{\partial eu}{\partial x} - P_g\frac{\partial u}{\partial x} + \frac{\partial}{\partial t} \kappa \frac{\partial}{\partial t} T_g. 
$$

This means that the fluxes can be written as $F_\rho = \rho u$, $F_v = \rho u^2 + P_g$, and $F_e = u(e + P_g)$. 

These are implemented in `step_density`, `step_momentum` and `step_energy`. 

Next, we evolve the equations in time in `evolv_hydro`. 

The pressure, $P_g = (\gamma - 1 )e\rho$ is the extra energy-pressure relation where $\gamma = 5/3$, and $u = p/\rho$, where $p$ is the momentum. 


__Challenge__: Consider a Super-Time-Stepping method [See Ex_5](https://gitlab.com/ast5110_course/ast5110/-/blob/master/ex_5a.ipynb) and [see also Nobrega-Siverio et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...638A..79N/abstract)


</span>

In [ ]:
def evol_hd_sts(xx, rho, u, Tg, gamma=5/3, kappa=0, kB=1, mH=1, nt=1000, cfl_cut=0.48, \
                    ddx=nm.deriv_cent, bnd_type='wrap', bnd_limits=[1,1], nu=0.9, n_sts=10, S=lambda x, t: 0, **kwargs): 
    """
    Evolve the hydrodynamical system nt steps in time, 
    including thermal conduction and a possible source term 
    with the super timestepping method.

    Parameters
    ----------
        xx : `array`
            Spatial axis. 
        rho : `array`
            Initial density.
        u : `array`
            Initial velocity.
        Tg : `array`
            Initial temperature.
        gamma : `float`
            Adiabatic index.
        kappa : `float`
            Thermal conduction coefficient.
        kb : `float`
            Boltzmann constant.
        mh : `float`
            Mass of hydrogen atom.
        nt : `int`
            Number of timesteps.
        cfl_cut : `float`
            CFL condition to limit the timestep. 
        ddx : `function`
            Derivative method.
        bnd_type : `str`
            Boundary type.
        bnd_limits : `list`
            Boundary limits.
        nu : `float`
            Dampening factor between 0 and 1
        n_sts : `int`
            Number of super timesteps, positive 
        S : `function`
            Source term.

    Returns
    -------
        t : `array`
            Time axis.
        rho_arr : `array`
            Density evolution.
        mom_arr : `array`
            Momentum evolution.
        e_arr : `array`
            Energy evolution.
    """

    eps = 1e-6 # to avoid division by zero

    ## Declare the arrays ##
    t = np.zeros((nt))                # time
    rho_arr = np.zeros((len(xx), nt)) # density
    mom_arr = np.zeros((len(xx), nt)) # momentum
    e_arr1  = np.zeros((len(xx), nt)) # energy without thermal conduction
    e_arr2  = np.zeros((len(xx), nt)) # energy with thermal conduction
    e_arr   = np.zeros((len(xx), nt)) # energy
    
    ## Initialize variables ##
    rho_arr[:,0] = rho   # Ok
    mom_arr[:,0] = rho*u # u = p/rho OK
    Pg           = rho*kB*Tg / mH # Pg = n*kB*Tg -> n = rho/mH # OK
    # e_arr[:,0] = Pg/(gamma - 1)    # internal energy           # XXX
    e_arr[:,0] = Pg/(gamma - 1)*rho    # internal energy           # XXX

    ## Evolve the system in time ##
    for i in range(nt - 1): 
        
        ## Update variables ##
        u  = mom_arr[:, i]/(rho_arr[:, i] + eps)              # u = p/rho OK
        # Tg = e_arr[:, i]*mH*(gamma - 1) / (kB*rho_arr[:,i])   # XXX
        Tg = e_arr[:, i]*mH*(gamma - 1) / (kB*rho_arr[:,i]**2)   # XXX
        Pg = rho_arr[:, i]*kB*Tg / mH                         # OK
        cs = np.sqrt(np.abs(gamma*Pg / (rho_arr[:,i] + eps))) # sound speed 
        S_arr = S(xx, i)

        ## CFL condition ##
        """ Note: Defining the sound speed might be enough for now ,but u might matter more later in MHD """
        dt1 = np.min(np.gradient(xx)    / np.abs(u + eps))         # Eigenvalue u 
        dt2 = np.min(np.gradient(xx)    / np.abs(u - cs + eps))    # Eigenvalue u-c_s
        dt3 = np.min(np.gradient(xx)    / np.abs(u + cs + eps))    # Eigenvalue u+c_s
        dt4 = np.min(np.gradient(xx)**2 / np.abs((2*kappa) + eps)) # Thermal conduction term
        
        # How fast the information can travel in that medium
        dt_cfl = np.min(np.array([dt1, dt2, dt3, dt4]))
        dt_sts = nm.tau_sts(nu, n_sts, dt_cfl)

        dt = cfl_cut*np.min(np.array([dt_cfl, dt_sts]))    # Final timestep

    ### Time evolution for density and momentum (NO SPLIT) ###

        ## Spatially averaged variables ##
        rhoLAX = (np.roll(rho_arr[:,i], -1) + rho_arr[:,i] + np.roll(rho_arr[:,i], 1)) / 3
        momLAX = (np.roll(mom_arr[:,i], -1) + mom_arr[:,i] + np.roll(mom_arr[:,i], 1)) / 3

        ## RHS of the HD equations ##
        # _, rhs_cont = step_density( xx, rho_arr[:,i], u,        axis=0, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits) # continuety eq
        # _, rhs_mom  = step_momentum(xx, mom_arr[:,i], u, Pg=Pg, axis=0, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits) # momentum eq
        rhs_cont = - ddx(xx, rho_arr[:,i] * u) # continuity equation
        g=0
        rhs_mom  = - ddx(xx, mom_arr[:,i] * u + Pg) - rho_arr[:,i] * g  # momentum balance

        rho_tmp = rhoLAX + rhs_cont*dt # Density 
        mom_tmp = momLAX + rhs_mom*dt  # Momentum
            
        # Boundaries: 
        if bnd_limits[1] > 0:  # downwind or centered scheme
            rhoBC = rho_tmp[bnd_limits[0]: - bnd_limits[1]]
            momBC  = mom_tmp[bnd_limits[0]: - bnd_limits[1]]
        else:
            rhoBC = rho_tmp[bnd_limits[0]:]  # upwind
            momBC  = mom_tmp[bnd_limits[0]:]
        
        ## Update arrays witn boundary conditions ##
        rho_arr[:,i+1] = np.pad(rhoBC, bnd_limits, bnd_type)
        mom_arr[:,i+1] = np.pad(momBC, bnd_limits, bnd_type)

    ### Time evolution for energy (OPERATOR SPLITTING) ###
        if np.abs(kappa) > 0: # No thermal conduction
            
            ## Spatially averaged variables ## 
            eLAX1 = (np.roll(e_arr[:,i], -1) + e_arr[:,i] + np.roll(e_arr[:,i], 1)) / 3

            ## RHS of the HD equations ##
            # _, rhs_energy = step_energy(xx, e_arr[:,i], u, Pg=Pg, axis=0, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits) # energy eq
            rhs_energy1 =  - ddx(xx, e_arr[:,i] * u) - Pg * ddx(xx, u) + S_arr # energy balance 1

            e_tmp1 = eLAX1 + rhs_energy1*dt # Energy

            # Boundaries:
            if bnd_limits[1] > 0:  # downwind or centered scheme
                eBC1 = e_tmp1[bnd_limits[0]: - bnd_limits[1]]
            else:
                eBC1 = e_tmp1[bnd_limits[0]:]
            
            ## Update arrays witn boundary conditions ##
            e_arr1[:,i+1] = np.pad(eBC1, bnd_limits, bnd_type)

            ### Super timesteppig for thermal conduction ###

            ## Declare the arrays ##
            e_sts = np.zeros((np.size(xx), n_sts))
            t_sts = np.zeros((n_sts))

            ## Initialize variables ##
            e_sts[:,0] = e_arr[:,i]

            ## Evolve the system in time ##
            for j in range(n_sts - 1):
                ## Initialize variables based on the previous super-timesteps ##
                dtj = dt_cfl * nm.taui_sts(nu, n_sts, j)
                Tg_sts = e_sts[:,j]*mH*(gamma - 1) / kB # XXX 
                
                ## Spatially averaged variables ##
                eLAX2 = (np.roll(e_sts[:,j], -1) + e_sts[:,j]  + np.roll(e_sts[:,j], 1)) / 3

                ## RHS of the HD equations ##
                # _, rhs_energy = step_energy(xx, e_arr[:,i], u, Pg=Pg, axis=0, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits) # energy eq
                rhs_energy2 = kappa * ddx(xx, ddx(xx, Tg_sts)) 

                e_tmp2 = eLAX2 + rhs_energy2*dtj # Energy

                # Boundaries:
                if bnd_limits[1] > 0:  # downwind or centered scheme
                    eBC2 = e_tmp2[bnd_limits[0]: - bnd_limits[1]]
                else:
                    eBC2 = e_tmp2[bnd_limits[0]:]
                
                ## Update arrays witn boundary conditions ##
                e_sts[:,j+1] = np.pad(eBC2, bnd_limits, bnd_type)
                t_sts[j+1] = t_sts[j] + dtj
            
            ## Variable update for normal stepping XXX ##
            e_arr2[:,i+1] = e_sts[:, n_sts-1]

            ## Adding the advances in the operator split to get the full time advance ##
            e_arr[:,i+1] = e_arr1[:,i+1] + e_arr2[:,i+1] - ((np.roll(e_arr[:,i],1) + e_arr[:,i]+ np.roll(e_arr[:,i],-1)) / 3)
            t[i+1] = t[i] + np.sum(dt)

        else:                 # Thermal conduction and source term

            ## Spatially averaged variables ## 
            eLAX = (np.roll(e_arr[:,i], -1) + e_arr[:,i] + np.roll(e_arr[:,i], 1)) / 3

            ## RHS of the HD equations ##
            # _, rhs_energy = step_energy(xx, e_arr[:,i], u, Pg=Pg, axis=0, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits) # energy eq
            rhs_energy = -ddx(xx, e_arr[:,i]*u) - Pg*ddx(xx, u) + S_arr

            e_tmp = eLAX + rhs_energy*dt # Energy
            
            # Boundaries:
            if bnd_limits[1] > 0:  # downwind or centered scheme
                eBC = e_tmp[bnd_limits[0]: - bnd_limits[1]]
            else:
                eBC = e_tmp[bnd_limits[0]:]
            
            ## Update arrays witn boundary conditions ##
            e_arr[:,i+1] = np.pad(eBC, bnd_limits, bnd_type)
            t[i+1] = t[i] + dt

    return t, rho_arr, mom_arr, e_arr

### Testing the code 

In order to test the code, we initialize a Gaussian density pertubation. 

<span style="color:red">

TODO: Check what this is: 
Use the same asymptotical solutions to test the thermal conduction term as in [Ex_5](https://gitlab.com/ast5110_course/ast5110/-/blob/master/ex_5a.ipynb)

</span>

In [ ]:
def gauss(xx, a, x0, b):
    return a*np.exp(-(xx-x0)**2/(b**2)) + 1 # XXX TODO: check if this is correct

## Initialize constants ##
gamma = 5/3
mh = 1 
kb = 1

## Initialize the grid ##
xi = 0
xf = 4
x_half = xf/2
numps = 512
dx = (xf-xi) / (numps) 
xx = np.arange(xi, xf, dx)

## Initialize the initial conditions ##
rho0 = gauss(xx, a=0.05, x0=x_half, b=0.1)
u0 = 0.0001*np.ones(numps)
Tg0 = 1*np.ones(numps)
Pg0 = rho0*kb*Tg0 / mh
e0 = (rho0 * kb * Tg0) / (mh*(gamma-1))


<span style="color:pink">


__Suggestion__ This test might be too complex to debug the code. However, as a starting debugging test, you can consider an advection test (constant initial velocity) of a gaussian density perturbation in pressure balance. 

To start, we plot the initial conditions to chech that the function is initialized correctly: 

</span>

In [ ]:
## Plot the initial conditions ##

def plot_hd_init(xx, density, velocity, temperature, pressure, energy): 
    """ 
    Function for plotting the initial values for the density, velocity, 
    temperature, pressure and energy. 

    Parameters 
    ----------
    xx : `array`
        Spatial axis. 
    density: `array`
        1D array of the density.
    velocity: `array`
        1D array of the velocity.
    temperature: `array`
        1D array of the temperature.
    pressure: `array`
        1D array of the pressure.
    energy: `array`
        1D array of the energy.

    Returns
    -------
        A 6 panel plot of the initial values for the density, velocity, 
        temperature, pressure and energy. 
    """
    dpi = 200 

    fig, ax = plt.subplots(3, 2, figsize=(12,12), dpi=dpi)

    ax[0,0].plot(xx, density, color='mediumpurple', label="Density")
    ax[0,0].set_xlabel("x", fontsize=14)
    ax[0,0].grid()
    ax[0,0].set_title("Density", fontsize=14)

    ax[0,1].plot(xx, velocity, color='green', label="Velocity")
    ax[0,1].set_xlabel("x", fontsize=14)
    ax[0,1].grid()
    ax[0,1].set_title("Velocity", fontsize=14)

    ax[1,0].plot(xx, temperature, color='orange', label="Temperature")
    ax[1,0].set_xlabel("x", fontsize=14)
    ax[1,0].grid()
    ax[1,0].set_title("Temperature",fontsize=14)

    ax[1,1].plot(xx, pressure, color='blue', label="Pressure")
    ax[1,1].set_xlabel("x", fontsize=14)
    ax[1,1].grid()
    ax[1,1].set_title("Pressure",fontsize=14)

    ax[2,0].plot(xx, energy, color='cyan', label="Energy")
    ax[2,0].set_xlabel("x", fontsize=14)
    ax[2,0].grid()
    ax[2,0].set_title("Energy",fontsize=14)

    ax[2,1].plot(xx, density, color='mediumpurple', label="density")
    ax[2,1].plot(xx, velocity, color='green', label="velocity")
    ax[2,1].plot(xx, temperature, color='orange', label="temperature")
    ax[2,1].plot(xx, pressure, color='blue', linestyle='dotted', label="pressure")
    ax[2,1].plot(xx, energy, color='cyan', label="energy")
    ax[2,1].set_xlabel("x", fontsize=14)
    ax[2,1].grid()
    ax[2,1].set_title("All", fontsize=14)
    ax[2,1].legend(loc='lower right', fontsize=14)

    plt.tight_layout()


Next, we evolve the discontinuous numerical scheme and plot the results: 

## 4- (nano)-flares in the corona.  

Add an ad-hoc source term: 

$ \frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \bf u) = 0 $

$ \frac{\partial \rho {\bf u}}{\partial t} + \nabla \cdot (\rho {\bf u}  \otimes {\bf u}) = - \nabla (P_g)$

$ \frac{\partial e}{\partial t } = -\nabla\cdot e {\bf u} -P_g \nabla \cdot {\bf u} + \nabla \kappa \nabla T_g + S(x,t)$

where $S(x,t)$ is the ad-hoc source term which depend in space and time and it will have the follosing description: 

$S(x,t) = A \exp(-(x-x_0)^2/W_s^2) \exp(-(t-t_0)^2/W_t^2)$

where $A$ is the amplitud, $W_s$ the spatial width of the Gaussian centered at $x_0$, and $W_t$ the temporal range of the Gaussian centered at $t_0$. Consider different $W_s$ ranging from a few tens of km to a few Mm and $W_t$ ranging from a few fraction of second a 100s. 

For the initial condition: implement the same initial scenario as in [Heggland et al. 2007](https://iopscience.iop.org/article/10.1086/518828/pdf). Consider various density/temperature levels in the corona. 

One of the candidates to heat the corona is due to braiding of the magnetic field. Braiding can produce episodic (flaring) and localized heating in the solar atmosphere which is propagated along the magnetic field lines via thermal conduction or electron beans. The energy released on these flaring events can go from nano-flares to class X flares. Consider a 1D stratified atmosphere and invesgate the energy deposition along the 1D atmosphere for various background stratifications and energies for the flaring events. Where are you going to put the source? How does it travel? Compare with what we have learned from the assymptotic solution in [Ex_5](https://gitlab.com/ast5110_course/ast5110/-/blob/master/ex_5a.ipynb). Can you see evaporation? When? 

The parameter range is huge for this exercise ($W_s$, $W_t$, A, and initial conditions). Play with one thing at a time. No need to do all possible combinations. 

__Bonus__ consider a more irregular source in time: 

$S(x,t) = \sum_i^n A_i \exp(-(x-x_i)^2/W_{si}^2) \exp(-(t-t_i)^2/W_{ti}^2)$



In [ ]:
def S(x, t): 
    A = 0.1 # Amplitude 
    x0 = 2 # Center of the gaussian spatial
    Ws = 0.1 # Spatial width of the gaussian
    Wi = 0.1 # Temporal range of the gaussian 
    t0 = 0 # Center of the gaussian temporal

    return A*np.exp(-(x - x0)**2 / Ws**2)*np.exp(-(t - t0)**2 / Wi**2)